<a href="https://colab.research.google.com/github/mfavaits/YouTube-Series-on-Machine-Learning/blob/master/Anomaly_Detection_Credit_Card_Fraud.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import numpy as np #linear algebra library of Python
import pandas as pd # build on top of numpy for data analysis, data manipulation and data visualization

Now let's mount Google drive so that we can upload the diabetes.csv file. You can find the code in the 'Code snippets' tab of Colab

In [0]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


First thing that we do is take a look at the shape of the dataframe (df.shape) and take a look at first 5 lines through df.head()

In [0]:
df=pd.read_csv('/content/gdrive/My Drive/Colab Notebooks/creditcard.csv') #import file from Google Drive and create a pandas dataframe df
df.head() #shows first 5 lines including column namesdf.shape # number of rows and columns of dataframe

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,V11,V12,V13,V14,V15,V16,V17,V18,V19,V20,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
0,0.0,-1.359807,-0.072781,2.536347,1.378155,-0.338321,0.462388,0.239599,0.098698,0.363787,0.090794,-0.551600,-0.617801,-0.991390,-0.311169,1.468177,-0.470401,0.207971,0.025791,0.403993,0.251412,-0.018307,0.277838,-0.110474,0.066928,0.128539,-0.189115,0.133558,-0.021053,149.62,0
1,0.0,1.191857,0.266151,0.166480,0.448154,0.060018,-0.082361,-0.078803,0.085102,-0.255425,-0.166974,1.612727,1.065235,0.489095,-0.143772,0.635558,0.463917,-0.114805,-0.183361,-0.145783,-0.069083,-0.225775,-0.638672,0.101288,-0.339846,0.167170,0.125895,-0.008983,0.014724,2.69,0
2,1.0,-1.358354,-1.340163,1.773209,0.379780,-0.503198,1.800499,0.791461,0.247676,-1.514654,0.207643,0.624501,0.066084,0.717293,-0.165946,2.345865,-2.890083,1.109969,-0.121359,-2.261857,0.524980,0.247998,0.771679,0.909412,-0.689281,-0.327642,-0.139097,-0.055353,-0.059752,378.66,0
3,1.0,-0.966272,-0.185226,1.792993,-0.863291,-0.010309,1.247203,0.237609,0.377436,-1.387024,-0.054952,-0.226487,0.178228,0.507757,-0.287924,-0.631418,-1.059647,-0.684093,1.965775,-1.232622,-0.208038,-0.108300,0.005274,-0.190321,-1.175575,0.647376,-0.221929,0.062723,0.061458,123.50,0
4,2.0,-1.158233,0.877737,1.548718,0.403034,-0.407193,0.095921,0.592941,-0.270533,0.817739,0.753074,-0.822843,0.538196,1.345852,-1.119670,0.175121,-0.451449,-0.237033,-0.038195,0.803487,0.408542,-0.009431,0.798278,-0.137458,0.141267,-0.206010,0.502292,0.219422,0.215153,69.99,0


In [0]:
df.shape # provides # rows and # columns of the dataframe df - 768 rows and 9 columns

(284807, 31)

Let's create numpy arrays, one for the features (X) and one for the label (y)



In [0]:
X=df.drop('Class', 1).values #drop 'Outcome' column but you keep the index column
y=df['Class'].values

We import the train_test_split function from sklearn to split the arrays or matrices into random train and test subsets>

Parameters:	
test_size : in our case 20% (default=0.25)

random_state: is basically used for reproducing your problem the same every time it is run. If you do not use a random_state in train_test_split, every time you make the split you might get a different set of train and test data points and will not help you in debugging in case you get an issue. We used random_state=42 but number does not matter

stratify : array-like or None (default=None)
If the number of values belonging to each class are unbalanced, using stratified sampling is a good thing. You are basically asking the model to take the training and test set such that the class proportion is same as of the whole dataset, which is the right thing to do.

In [0]:
random_state = np.random.RandomState(42)

In [0]:
from sklearn.model_selection import train_test_split #method to split training and testing data sets
X_train, X_test, y_train, y_test=train_test_split(X, y, test_size=0.2, random_state=random_state, stratify=y)

In [0]:
#from sklearn.preprocessing import StandardScaler 
#sc=StandardScaler()
#X_train=sc.fit_transform(X_train)
#X_test=sc.transform(X_test)

Now we are ready to use AD algorithm

In [0]:
pip install pyOD

In [0]:
from pyod.models.lof import LOF   # LOF detector
from pyod.models.ocsvm import OCSVM # One Class SVM
from pyod.models.iforest import IForest #Isolated Forests
from pyod.utils.data import evaluate_print

In [0]:
Classifiers = {'Local Outlier Factor (LOF)': LOF(n_neighbors=20, algorithm='auto', metric='minkowski', p=2, metric_params=None, contamination=0.002, n_jobs=-1), 
               'OneClassSVM': OCSVM (kernel='poly', degree=3 , nu=0.1, max_iter=-1, contamination=0.002), 
               'Isolation Forests': IForest(n_estimators=100, contamination=0.002, n_jobs=-1, random_state=random_state)}

In [0]:
for i, (clf_name, clf) in enumerate(Classifiers.items()):
        print()
        print(i + 1, 'fitting', clf_name)
        
        # fit the data and tag outliers
        clf.fit(X_train) # fit model on training data
       
        y_train_pred = clf.labels_ # the binary labels of the training data: 0 for inliers and 1 for anomalies
        y_train_scores = clf.decision_scores_ # the outlier scores of the training data
        
        y_test_pred = clf.predict(X_test) # predict if a particular sample is an outlier or not
        y_test_scores = clf.decision_function(X_test) # predict raw anomaly score of the transaction
        y_test_proba=clf.predict_proba(X_test) # predict the probability of a sample being outlier
        
        evaluate_print(clf_name, y_test, y_test_scores)


1 fitting Local Outlier Factor (LOF)
Local Outlier Factor (LOF) ROC:0.7673, precision @ rank n:0.051

2 fitting OneClassSVM
OneClassSVM ROC:0.6295, precision @ rank n:0.0

3 fitting Isolation Forests


/usr/local/lib/python3.6/dist-packages/sklearn/ensemble/iforest.py:247: FutureWarning: behaviour="old" is deprecated and will be removed in version 0.22. Please use behaviour="new", which makes the decision_function change to match other anomaly detection algorithm API.
  FutureWarning)


Isolation Forests ROC:0.9519, precision @ rank n:0.2551
